In [70]:
#https://anaconda.org/conda-forge/py2neo
#dbms.security.auth_enabled=false
from py2neo import Graph
graph = Graph(password="neo4j")
graph.data("MATCH (a:Person) RETURN a.name, a.born LIMIT 4")

[{'a.name': 'Charlize Theron', 'a.born': 1975},
 {'a.name': 'Al Pacino', 'a.born': 1940},
 {'a.name': 'Taylor Hackford', 'a.born': 1944},
 {'a.name': 'Tom Cruise', 'a.born': 1962}]

In [93]:
from pandas import DataFrame
DataFrame(graph.data("MATCH (a:Person) RETURN a.name, a.born  LIMIT 4"))

,a.born,a.name
0,1975,Charlize Theron
1,1940,Al Pacino
2,1944,Taylor Hackford
3,1962,Tom Cruise


In [17]:
#兩個節點間的關係
from py2neo import Node, Relationship
a = Node("Person", name="Jerry")
b = Node("Person", name="Bill")
ab = Relationship(a, "KNOWS", b)
ab

(Jerry)-[:KNOWS {}]->(Bill)

In [18]:
#對每一個節點與關係給值
a['age'] = 20
b['age'] = 21
ab['time'] = '2017/08/31'
print(a, b, ab)

(:Person {age: 20, name: 'Jerry'}) (:Person {age: 21, name: 'Bill'}) (Jerry)-[:KNOWS {time: '2017/08/31'}]->(Bill)


In [38]:
#子圖
from py2neo import Node, Relationship 
a = Node("Person", name="Jerry")
b = Node("Person", name="Bill")
ab = Relationship(a, 'KNOWS', b)
s = a | b | ab
print(s)

({(:Person {name: 'Jerry'}), (:Person {name: 'Bill'})}, {(Jerry)-[:KNOWS {}]->(Bill)})


In [39]:
print(s.nodes())
print(s.relationships())

frozenset({(:Person {name: 'Jerry'}), (:Person {name: 'Bill'})})
frozenset({(Jerry)-[:KNOWS {}]->(Bill)})


In [45]:
#用&取交集
s1 = a | b | ab
s2 = a | b
print(s1 & s2)

({(:Person {name: 'Jerry'}), (:Person {name: 'Bill'})}, {})


In [46]:
from py2neo import Node, Relationship, size, order
s = a | b | ab
print(s.keys()) #返回屬性集合
print(s.labels()) #返回所有Label的集合
print(s.nodes()) #返回所有節點的集合
print(s.relationships()) #子圖的關係集合
print(s.types()) #子圖的關係的type集合
print(order(s)) #子圖節點的數目
print(size(s)) #子圖的關係數目

frozenset({'name'})
frozenset({'Person'})
frozenset({(:Person {name: 'Jerry'}), (:Person {name: 'Bill'})})
frozenset({(Jerry)-[:KNOWS {}]->(Bill)})
frozenset({'KNOWS'})
2
1


In [47]:
from py2neo import Node, Relationship

a = Node('Person', name='Alice')
b = Node('Person', name='Bob')
c = Node('Person', name='Mike')
ab = Relationship(a, "KNOWS", b)
ac = Relationship(a, "KNOWS", c)
w = ab + Relationship(b, "LIKES", c) + ac
print(w)

(Alice)-[:KNOWS {}]->(Bob)-[:LIKES {}]->(Mike)<-[:KNOWS {}]-(Alice)


In [48]:
#遍歷的方式
from py2neo import walk

for item in walk(w):
    print(item)

(:Person {name: 'Alice'})
(Alice)-[:KNOWS {}]->(Bob)
(:Person {name: 'Bob'})
(Bob)-[:LIKES {}]->(Mike)
(:Person {name: 'Mike'})
(Alice)-[:KNOWS {}]->(Mike)
(:Person {name: 'Alice'})


In [ ]:
#RNN + Neo4j

In [ ]:
#為了方便後面做Index用，建立a-z,A-Z的26*2個英文字母，加上5個特殊符號，共57個符號
import string
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

#建立函式方便尋找all_letters的Index像是"a"=10000000000....
def letterToIndex(letter):
    return all_letters.find(letter)

# 將一個字母轉成Tensor顯示
def letterToTensor(letter):
    tensor = np.zeros((1, n_letters)) #建立一個1*57的Tensor
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# 建立one-hot letter vectors
def lineToTensor(line):
    tensor = np.zeros((len(line), 1, n_letters))
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

#print(letterToTensor('b')) #將內容轉為字母 #ex. a,b,c..
#print(lineToTensor('Jones')) #5為字母個數、1為維度、57為總字母

In [342]:
from pandas import DataFrame
k = DataFrame(graph.data("MATCH (a:Person) RETURN a.name, a.born"))
#k[ k['a.name'] == 'Charlize Theron' ]
#k_unique = k['a.born'].unique()

In [ ]:
a = lineToTensor(k['a.name'][number][0])[0][0]
for number in range(1,len(k['a.name'])):
    ltt = lineToTensor(k['a.name'][number][0])
    b = ltt[0][0]
    a = np.vstack((a, b))

In [352]:
import tensorflow as tf
import numpy as np

# Mini-batch: instance 0,instance 1,instance 2,instance 3
#X0_batch = np.array([[0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0]]) # t = 0
#X1_batch = np.array([[0, 1, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0]]) # t = 1
X0_batch = a[0:133]
X1_batch = a[133:266]

n_inputs = 57
n_neurons = 5
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])
Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons],dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons,n_neurons],dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))
Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})
    print(Y0_val)
    print(Y1_val)

[[-0.8935173  -0.04895241  0.28290287 -0.36209202  0.53899896]
 [-0.8935173  -0.04895241  0.28290287 -0.36209202  0.53899896]
 [ 0.33245322  0.6619485   0.9066488   0.97798896 -0.33848122]
 [ 0.33245322  0.6619485   0.9066488   0.97798896 -0.33848122]
 [-0.43792397 -0.8215508  -0.8408758   0.42629457 -0.8564556 ]
 [-0.2917301  -0.94594926  0.67996305 -0.5251192   0.73472655]
 [-0.45196313  0.6255262  -0.09037119 -0.0059572  -0.8691708 ]
 [-0.45196313  0.6255262  -0.09037119 -0.0059572  -0.8691708 ]
 [-0.7824683  -0.9109145  -0.90518457  0.2266298  -0.34160444]
 [-0.00547187  0.8060577   0.2891354   0.37114635 -0.37079656]
 [-0.45196313  0.6255262  -0.09037119 -0.0059572  -0.8691708 ]
 [-0.43792397 -0.8215508  -0.8408758   0.42629457 -0.8564556 ]
 [-0.43792397 -0.8215508  -0.8408758   0.42629457 -0.8564556 ]
 [-0.00547187  0.8060577   0.2891354   0.37114635 -0.37079656]
 [ 0.8967452   0.9639617  -0.9885045  -0.4789405   0.09279011]
 [-0.8935173  -0.04895241  0.28290287 -0.36209202  0.53

In [109]:
Y1_val.max()

0.96133494